In [3]:
import os
import numpy as np
import soundfile as sf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from ssqueezepy import ssq_cwt
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import os
import numpy as np
np.complex = complex  # Fix for deprecated np.complex
import librosa
from ssqueezepy import ssq_cwt
import tqdm

# === Define paths and labels ===
base_dir = "/content/drive/MyDrive/signal_analysis/data"
class_labels = ['AS', 'MR', 'MS', 'MVP', 'N']

# === Function to process dataset with SST ===
def process_dataset_with_sst(base_dir, class_labels, sampling_rate=2000, fixed_length=300):
    X_sst = []
    y_labels = []

    for idx, label in enumerate(class_labels):
        folder_path = os.path.join(base_dir, label)

        if not os.path.exists(folder_path):
            print(f"Skipping missing folder: {folder_path}")
            continue

        for file_name in tqdm.tqdm(os.listdir(folder_path), desc=f"Processing {label}"):
            if file_name.endswith('.wav'):
                try:
                    file_path = os.path.join(folder_path, file_name)

                    # === Load and resample audio ===
                    signal, sr = librosa.load(file_path, sr=None)
                    signal = librosa.resample(signal, orig_sr=sr, target_sr=sampling_rate)
                    signal = signal / (np.max(np.abs(signal)) + 1e-8)

                    # === Apply SST ===
                    Tx, *_ = ssq_cwt(signal, fs=sampling_rate, nv=16, transform='cwt', padtype='symmetric')

                    # === Feature Extraction ===
                    sst_feature = np.mean(np.abs(Tx), axis=1)  # (freq_bins,)

                    # === Resize to fixed length ===
                    if len(sst_feature) < fixed_length:
                        sst_feature = np.pad(sst_feature, (0, fixed_length - len(sst_feature)), mode='constant')
                    else:
                        sst_feature = sst_feature[:fixed_length]

                    X_sst.append(sst_feature)
                    y_labels.append(idx)

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

    return np.array(X_sst), np.array(y_labels)

# === Process the dataset ===
X_sst, y_sst = process_dataset_with_sst(base_dir, class_labels)

# === Print shapes ===
print("X_sst shape:", X_sst.shape)
print("y_sst shape:", y_sst.shape)
print("Unique labels:", np.unique(y_sst))
